In [1]:
#Import all the libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

In [2]:
# Global variables for the code, can change planetary parameters here 
def defaults():
    global jmxdef, runlengthdef, scaleQdef, Adef, Bdef, Dmagdef, nudef
    global Cldef, Cwdef, coldstartdef, hadleyflagdef, albedoflagdef
    global obldef, eccdef, perdef, ice_modeldef, landdef, casenamedef
    # Default values (adjust as needed)
    jmxdef = 360
    runlengthdef = 300
    scaleQdef = 0.55
    Adef = 203.3
    Bdef = 2.08
    Dmagdef = 0.44
    nudef = 3
    Cldef = 0.45
    Cwdef = 9.8
    coldstartdef = 0
    hadleyflagdef = 1.0
    albedoflagdef = 0.0
    obldef = 0
    eccdef = 0
    perdef = 102.0651294475608
    ice_modeldef = 0
    landdef = 'modern'
    casenamedef = 'Control'

In [3]:
# --------------------------------------------------------------------
# seasonal_solar: Computes the insolation and day offset (from MATLAB code)
# --------------------------------------------------------------------
def seasonal_solar(xi, obl, ecc, lon):
    xi = np.asarray(xi)
    npts = len(xi)
    t1 = 2808 / 2.0754
    dr_conv = np.pi / 180.0
    rd_conv = 1 / dr_conv
    lmr = 0.0  # reference day (March 21)
    beta = np.sqrt(1 - ecc**2)
    lm0 = lmr - 2 * (
        (0.5 * ecc + 0.125 * ecc**3) * (1 + beta) * np.sin(lmr - lon * dr_conv)
        - 0.25 * ecc**2 * (0.5 + beta) * np.sin(2 * (lmr - lon * dr_conv))
        + 0.125 * ecc**3 * ((1/3) + beta) * np.sin(3 * (lmr - lon * dr_conv))
    )
    it = 360
    lm = np.linspace(lm0, lm0 + 2 * np.pi - 2 * np.pi / it, it)
    calendarlongitude = (
        lm
        + (2 * ecc - 0.25 * ecc**3) * np.sin(lm - lon * dr_conv)
        + (5/4) * (ecc**2) * np.sin(2 * (lm - lon * dr_conv))
        + (13/12) * (ecc**3) * np.sin(3 * (lm - lon * dr_conv))
    )
    calendarlongitude = np.mod(calendarlongitude, 2 * np.pi)
    lme = np.concatenate((lm, lm, lm))
    calendarlongitudee = np.concatenate((
        calendarlongitude - 2 * np.pi,
        calendarlongitude,
        calendarlongitude + 2 * np.pi
    ))
    nts = 360
    t = np.linspace(1, 360, nts)
    tl = (t - 90) * dr_conv
    tl = np.mod(tl, 2 * np.pi)
    sort_idx = np.argsort(calendarlongitudee)
    f_interp = interp1d(calendarlongitudee[sort_idx], lme[sort_idx],
                        kind='linear', fill_value="extrapolate")
    astrolon = f_interp(tl)
    astroloni = 90 + (180 / np.pi) * astrolon
    astroloni = np.mod(astroloni, 360)
    dayoffset = (astroloni - t) * 365 / 360
    ti = astroloni
    distance = (1 - ecc**2) / (1 + ecc * np.cos(dr_conv * (450 + ti - lon)))
    s_delt = -np.sin(dr_conv * obl) * np.cos(dr_conv * ti)
    c_delt = np.sqrt(1.0 - s_delt**2)
    t_delt = s_delt / c_delt
    delt = np.arcsin(s_delt) * rd_conv  # in degrees
    phi = np.arcsin(xi) * rd_conv         # in degrees
    wk = np.zeros((npts, nts))
    for i in range(nts):
        for j in range(npts):
            if delt[i] > 0.0:
                if phi[j] >= 90 - delt[i]:
                    wk[j, i] = t1 * xi[j] * s_delt[i] / (distance[i]**2)
                elif (-phi[j] >= (90 - delt[i])) and (phi[j] < 0):
                    wk[j, i] = 0
                else:
                    c_h0 = -np.tan(dr_conv * phi[j]) * t_delt[i]
                    c_h0 = np.clip(c_h0, -1, 1)
                    h0 = np.arccos(c_h0)
                    wk[j, i] = t1 * (h0 * xi[j] * s_delt[i] +
                                     np.cos(dr_conv * phi[j]) * c_delt[i] * np.sin(h0)) \
                              / (distance[i]**2 * np.pi)
            else:
                if phi[j] >= (90 + delt[i]):
                    wk[j, i] = 0
                elif (-phi[j] >= (90 + delt[i])) and (phi[j] < 0):
                    wk[j, i] = t1 * xi[j] * s_delt[i] / (distance[i]**2)
                else:
                    c_h0 = -np.tan(dr_conv * phi[j]) * t_delt[i]
                    c_h0 = np.clip(c_h0, -1, 1)
                    h0 = np.arccos(c_h0)
                    wk[j, i] = t1 * (h0 * xi[j] * s_delt[i] +
                                     np.cos(dr_conv * phi[j]) * c_delt[i] * np.sin(h0)) \
                              / (np.pi * distance[i]**2)
    insol = wk
    return insol, dayoffset

In [7]:
#ice balance 
import numpy as np

def ice_balance(jmx, ice, notice, h, conduct, Tfrz, r, rprimew, Cw_delt, 
                M, Diff_Op, B, nu_fw, fw, delx):
    """
    Compute the ice balance and adjust the heat flux and ocean temperatures.

    Parameters
    ----------
    jmx : int
        Number of spatial grid cells.
    ice : ndarray of int
        1D array of grid-cell indices (0-indexed) where ice is present.
    notice : ndarray of int
        1D array of grid-cell indices (0-indexed) where no ice is present.
    h : ndarray, shape (jmx,)
        Ice thickness for each grid cell.
    conduct : float
        Thermal conductivity.
    Tfrz : float
        Freezing temperature.
    r : ndarray, shape (2*jmx,)
        Forcing vector (interleaved land and water components).
    rprimew : ndarray, shape (jmx,)
        Source term for water temperatures.
    Cw_delt : float
        A scaling constant (for water).
    M : ndarray, shape (2*jmx, 2*jmx)
        A matrix used in the temperature update.
    Diff_Op : ndarray, shape (jmx, jmx)
        Diffusion operator (applied to the water temperature field).
    B : float
        A scalar coefficient.
    nu_fw : ndarray, shape (jmx,)
        Parameter for the freshwater flux (for each grid cell).
    fw : ndarray, shape (jmx,)
        Grid-cell areas (or weighting factors) for the ice.
    delx : float
        Grid-box length scale.

    Returns
    -------
    T : ndarray, shape (2*jmx,)
        Updated full temperature vector (interleaved: land, water).
    L : ndarray, shape (jmx,)
        Land temperature (extracted from T).
    W : ndarray, shape (jmx,)
        Water temperature (extracted from T and adjusted below).
    Fnet : ndarray, shape (jmx,)
        Net heat flux for each grid cell.
    """

    # --- Part 1: Solve the temperature system with sea ice effects ---

    # Initialize k and Fnet as vectors of length jmx.
    k = np.zeros(jmx)
    Fnet = np.zeros(jmx)

    # For grid cells with ice, update k.
    # (Here h[ice] is assumed nonzero.)
    k[ice] = conduct / h[ice]

    # Update r for the water temperature component corresponding to ice.
    # In MATLAB, r(2*ice) (with 1-indexing) becomes r[2*i+1] in Python.
    r[2 * ice + 1] = k[ice] * Tfrz - rprimew[ice]

    # Create a deviation vector for the full 2*jmx temperature vector.
    dev = np.zeros(2 * jmx)
    dev[2 * ice + 1] = -Cw_delt + k[ice]

    # Form the modified matrix Mt = M + diag(dev)
    Mt = M + np.diag(dev)

    # Solve for the full temperature vector, I, such that Mt * I = r.
    I_full = np.linalg.solve(Mt, r)

    # Let T be I_full, but then adjust the water component for grid cells with ice.
    T = I_full.copy()
    # For each ice grid cell, water temperature index is 2*i+1.
    T[2 * ice + 1] = np.minimum(Tfrz, I_full[2 * ice + 1])

    # Extract land and water temperatures from the interleaved T vector.
    L = T[0::2]  # land temperatures (indices 0,2,4,…)
    W = T[1::2]  # water temperatures (indices 1,3,5,…)

    # For the flux calculation below, we need the water temperature vector.
    # (After the above adjustment, I_water is the same as W.)
    I_water = W.copy()

    # Compute the net flux Fnet for grid cells with ice.
    # Diff_Op[ice, :] is a submatrix (rows for ice indices); its dot product with the full
    # water temperature vector gives a contribution that is then adjusted.
    Fnet[ice] = (np.dot(Diff_Op[ice, :], I_water) - rprimew[ice] -
                 B * W[ice] - nu_fw[ice] * (I_water[ice] - L[ice]))

    # --- Part 2: Compute the ocean heat flux adjustment ---

    # Separate grid cells into northern and southern hemispheres.
    # (These conditions have been adjusted for 0-indexing.
    #  Here we assume that grid cells with index >= (jmx//2 + 1) are northern, and those < jmx//2 are southern.)
    nhice = ice[ice >= (jmx // 2 + 1)]
    shice = ice[ice < (jmx // 2)]
    nhocn = notice[notice >= (jmx // 2 + 1)]
    shocn = notice[notice < (jmx // 2)]

    # Compute total ice area (weighted by fw) in each hemisphere.
    nhicearea = np.sum(fw[nhice])
    shicearea = np.sum(fw[shice])

    # Maximum possible ocean/ice area in each hemisphere.
    # In MATLAB, fw(jmx/2+1:jmx) corresponds to Python fw[jmx//2:] (if jmx is even).
    nhmax = np.sum(fw[jmx // 2:])
    shmax = np.sum(fw[:jmx // 2])

    # Compute the under-ice freshwater flux (fw) for each hemisphere.
    # This flux asymptotes to 2 W/m².
    nhfw = 2 * min(2 - 2 * (nhicearea - delx) / nhmax, 2)
    shfw = 2 * min(2 - 2 * (shicearea - delx) / shmax, 2)

    # Add the freshwater flux to the net flux Fnet in ice-covered grid cells.
    Fnet[nhice] += nhfw
    Fnet[shice] += shfw

    # Compute the ice-free ocean area in each hemisphere.
    nhocnarea = nhmax - nhicearea
    shocnarea = shmax - shicearea

    # Compute the water temperature adjustment.
    nhdW = nhfw * nhicearea / nhocnarea / Cw_delt
    shdW = shfw * shicearea / shocnarea / Cw_delt

    # Adjust the ocean (water) temperature in the northern and southern ocean grid cells.
    W[nhocn] -= nhdW
    W[shocn] -= shdW

    # Reassemble the full temperature vector T (if needed, T has not been reassembled here,
    # but L and W are our main outputs along with Fnet).
    # One might choose to reinterleave L and W into T if desired.

    return T, L, W, Fnet


In [8]:

def albedo_seasonal(L, W, x):
    """
    Recalculate albedo for land and water.

    Parameters
    ----------
    L : array_like
        Array representing land-related values (e.g., temperature).
    W : array_like
        Array representing water-related values (e.g., temperature).
    x : array_like
        Array (e.g., latitude-transformed values) used in the albedo calculation.

    Returns
    -------
    alb_l : ndarray
        Calculated land albedo.
    alb_w : ndarray
        Calculated water albedo.
    """
    # Calculate base albedo values using elementwise operations
    alb_w = 0.31948 + 0.08 * (3 * x**2 - 1) / 2 - 0.05
    alb_l = 0.42480 + 0.08 * (3 * x**2 - 1) / 2 + 0.05

    # Override albedo values where conditions are met
    alb_w[W <= -2] = 0.51363
    alb_l[L <= -2] = 0.51363

    return alb_l, alb_w




In [9]:
# Albedo_seasonal_setup function is needed to run the seasonal file
# Assuming defaults and required variables are defined in another module or at the top of your script
# For example, the defaults like jmxdef, runlengthdef, etc., would be set before this.

# Function to initialize default parameters
def seasonal_setup():
    global jmx, runlength, scaleQ, A, B, Dmag, nu, Cl, Cw, coldstartflag, hadleyflag
    global albedoflag, obl, ecc, per, ice_model, land, casename, Tfrz, conduct, Lfice
    global ts, tf, nstepinyear, delt, nts, n_out, delx, x, xfull, phi, Toffset, L, W
    global D, fl, fw, insol, Cw_delt, Cl_delt, delt_Lf, nu_fw, nu_fl, lambda_, a, c, b
    global Diff_Op, bw, bl, Mw, Ml, M, invM, clim_alb_l, clim_alb_w

    # Load defaults (adjust these defaults based on your system)
    defaults()


    # Check and set values if not provided
    if 'jmx' not in globals(): jmx = jmxdef
    if 'runlength' not in globals(): runlength = runlengthdef
    if 'scaleQ' not in globals(): scaleQ = scaleQdef
    if 'A' not in globals(): A = Adef
    if 'B' not in globals(): B = Bdef
    if 'Dmag' not in globals(): Dmag = Dmagdef
    if 'nu' not in globals(): nu = nudef
    if 'Cl' not in globals(): Cl = Cldef
    if 'Cw' not in globals(): Cw = Cwdef
    if 'coldstartflag' not in globals(): coldstartflag = coldstartdef
    if 'hadleyflag' not in globals(): hadleyflag = hadleyflagdef
    if 'albedoflag' not in globals(): albedoflag = albedoflagdef
    if 'obl' not in globals(): obl = obldef
    if 'ecc' not in globals(): ecc = eccdef
    if 'per' not in globals(): per = perdef
    if 'ice_model' not in globals(): ice_model = ice_modeldef
    if 'land' not in globals(): land = landdef
    if 'casename' not in globals(): casename = casenamedef

    # Size of domain, ensure even number
    jmx = 2 * (jmx // 2)

    # Freezing temperature of ocean in degrees C
    Tfrz = -2
    conduct = 2  # conductivity of sea ice (for explicit sea ice)
    Lfice = 9.8 * 83.5 / 50  # Lf/Cp * depth of ML

    # Time loop parameters
    ts = 90  # First day of run is present day equinox
    tf = runlength - 0.25
    nstepinyear = 60
    delt = 1.0 / nstepinyear
    nts = int(np.floor(tf / delt))
    n_out = np.arange(1, nts + 1)

    # Set up x array
    delx = 2.0 / jmx
    x = np.arange(-1.0 + delx, 1.0, delx)
    xfull = np.arange(-1 + delx / 2, 1 - delx / 2, delx)
    phi = np.arcsin(xfull) * 180 / np.pi

    # Set up initial T profile
    if coldstartflag:
        Toffset = -40
    else:
        Toffset = 0.0
    L = 7.5 + 20 * (1 - 2 * xfull**2) + Toffset
    W = 7.5 + 20 * (1 - 2 * xfull**2) + Toffset

    # Heat diffusion coefficient
    if hadleyflag:
        D = Dmag * (1 + 9 * np.exp(-(x / np.sin(25 * np.pi / 180))**6))
    else:
        D = Dmag * np.ones_like(x)

    # Land fraction
    fl = 0.05 * np.ones(jmx)
    if 'Preca' in land[:5]:
        # Pangea (middle Cambrian)
        j = phi <= -45
        fl[j] = 0.3
        j = (phi > -45) & (phi <= 30)
        fl[j] = 0.5
    elif 'Ordov' in land[:5]:
        # Gondwanaland
        j = phi <= -60
        fl[j] = 0.95
        j = (phi > -60) & (phi <= 70)
        fl[j] = 0.3
    elif 'Symme' in land[:5]:
        # Symmetric
        fl.fill(0.01)
    else:  # Modern
        fl = 0.38 * np.ones(jmx)
        j = phi <= -60
        fl[j] = 0.95
        j = (phi > -60) & (phi <= -40)
        fl[j] = 0.05
        j = (phi > -40) & (phi <= 20)
        fl[j] = 0.25
        j = (phi > 20) & (phi <= 70)
        fl[j] = 0.5
    fl = fl * 0.01 / np.mean(fl)

    # Ocean fraction
    fw = 1 - fl

    # Obtain annual array of daily averaged insolation
    insol = seasonal_solar(xfull, obl, ecc, per)
    insol = scaleQ * insol[:, np.concatenate(([359], np.arange(360)))]  # Reorder to match

    # Set up deltas
    Cw_delt = Cw / delt
    Cl_delt = Cl / delt
    delt_Lf = delt / Lfice

    # Diffusion parameters
    nu_fw = nu / fw
    nu_fl = nu / fl
    lambda_ = D / delx / delx * (1 - x**2)
    a = np.concatenate(([0], -lambda_))
    c = np.concatenate(([-lambda_], [0]))
    b = -a - c
    Diff_Op = -(np.diag(b) + np.diag(c[:-1], 1) + np.diag(a[1:], -1))

    bw = Cw_delt + B + nu_fw - a - c
    bl = Cl_delt + B + nu_fl - a - c

    Mw = np.diag(bw) + np.diag(c[:-1], 1) + np.diag(a[1:], -1)
    Ml = np.diag(bl) + np.diag(c[:-1], 1) + np.diag(a[1:], -1)

    # Matrix M for implicit solution
    M = np.zeros((2 * jmx, 2 * jmx))
    for j in range(jmx):
        M[2 * j, 0::2 * jmx] = Ml[j, :]
        M[2 * j, 2 * j] = -nu_fl[j]
        M[2 * j + 1, 1::2 * jmx] = Mw[j, :]
        M[2 * j + 1, 2 * j - 1] = -nu_fw[j]

    invM = np.linalg.inv(M)

    # Climatological albedo if no albedo feedback
    if albedoflag:
        # Assuming temperatures.mat was previously loaded and processed in the seasonal setup
        clim_alb_l = np.zeros((jmx, 360))
        clim_alb_w = clim_alb_l.copy()
        n = 0
        for t in thedays:
            clim_alb_l[:, t], clim_alb_w[:, t] = albedo_seasonal(Lann[:, n], Wann[:, n], xfull)
            n += 1

# Calling this function will set up all the parameters and global variables
#seasonal_setup()


In [11]:

# import matplotlib.pyplot as plt   # if needed for plotting

# Assume that seasonal_setup, icebalance, albedo_seasonal, and plotoutput 
# are defined elsewhere and imported as needed.
# from my_module import seasonal_setup, icebalance, albedo_seasonal, plotoutput

# === Setup (equivalent to 'seasonal_setup;' in MATLAB) ===
seasonal_setup()  # This function should set up all necessary variables
# For example, it should define:
#   ice_model, jmx, W, Tfrz, albedoflag, clim_alb_l, clim_alb_w, thedays,
#   ts, insol, A, Cl_delt, Cw_delt, L, delt, nts, n_out, nstepinyear, 
#   delt_Lf, Fnet, invM, Lfice, etc.

# --- Initialize r as a 1D array of length 2*jmx ---
r = np.zeros(2 * jmx)

if ice_model:
    # set up initial sea ice thickness of 2 m where freezing occurs
    ice = np.where(W < Tfrz)[0]
    notice = np.where(W >= Tfrz)[0]
    h = np.zeros(jmx)       # sea ice thickness (for each spatial point)
    # k is defined in MATLAB but not used later so we skip it
    h[ice] = 2.0

    if albedoflag:
        # In MATLAB, the first element is thedays(1). In Python, use index 0.
        alb_l = clim_alb_l[:, thedays[0]]
        alb_w = clim_alb_w[:, thedays[0]]
    else:
        alb_l, alb_w = albedo_seasonal(L, W, xfull)

    # Calculate insolation using column 'ts' (assuming ts is already a proper index)
    S = insol[:, ts]

    rprimew = A - (1 - alb_w) * S
    rprimel = A - (1 - alb_l) * S

    # MATLAB sets odd entries (indices 1,3,5,...) for land and even entries for water.
    # In Python (0-indexed): r[0::2] are L (land) and r[1::2] are W (water).
    r[0::2] = L * Cl_delt - rprimel
    r[1::2] = W * Cw_delt - rprimew

    T, L, W, Fnet = ice_balance(jmx, ice, notice, h, conduct, Tfrz, r, rprimew,
                                 Cw_delt, M, Diff_Op, B, nu_fw, fw, delx)  # Update any sea ice–related variables
else:
    ice = np.array([])

# === Allocate output arrays ===
# L_out and W_out are jmx x (number of output times) arrays.
L_out = np.zeros((jmx, len(n_out)))
W_out = np.zeros((jmx, len(n_out)))
if ice_model:
    h_out = np.zeros((jmx, len(n_out)))

# Preallocate arrays to store time information (length = nts)
tday = np.zeros(nts)
yr   = np.zeros(nts, dtype=int)
day  = np.zeros(nts, dtype=int)

# --- Main time stepping loop ---
idx_out = 0  # Use 0-indexing for the output index
for i in range(nts):
    # In MATLAB, the loop index n goes from 1 to nts.
    n = i + 1  # adjust to 1-indexed n if needed for comparisons

    # Compute the day; MATLAB uses: tday(n)= ts +2 +1+(n-1)*360*delt
    tday[i] = ts + 3 + i * 360 * delt

    # Compute year and day-of-year (using floor division)
    yr[i]  = int(np.floor((tday[i] - 1) / 360))
    day[i] = int(np.floor(tday[i] - yr[i] * 360))

    # --- Create initial albedo ---
    if albedoflag:
        # Get index for albedo arrays.
        # MATLAB:
        #   nn = day(n);
        #   nn = nn - 360*delt; if (nn<0), nn = 360 - 360*delt * 0.5; end
        nn = day[i] - 360 * delt
        if nn < 0:
            nn = 360 - 360 * delt * 0.5
        nn = int(nn)  # ensure integer index
        alb_l = clim_alb_l[:, nn]
        alb_w = clim_alb_w[:, nn]
    else:
        alb_l, alb_w = albedo_seasonal(L, W, xfull)

    # --- Calculate insolation ---
    S = insol[:, day[i]]

    # --- Source terms ---
    rprimew = A - (1 - alb_w) * S
    rprimel = A - (1 - alb_l) * S

    r[0::2] = L * Cl_delt - rprimel
    r[1::2] = W * Cw_delt - rprimew

    if ice_model:
        # First, consider where sea ice already exists.
        ice_idx = np.where(h > 0.001)[0]
        notice_idx = np.where(h <= 0.001)[0]
        if ice_idx.size > 0:
            T, L, W, Fnet = ice_balance(jmx, ice, notice, h, conduct, Tfrz, r, rprimew,
                                 Cw_delt, M, Diff_Op, B, nu_fw, fw, delx)  # Update sea ice physics; this function should update Fnet, etc.
            h[ice_idx] = h[ice_idx] - delt_Lf * Fnet[ice_idx]
            h[ice_idx] = np.maximum(0.0, h[ice_idx])

        # Second, consider the conditions for new ice growth over the ocean.
        # In MATLAB, T(2*notice) selects the water temperature for indices corresponding to notice.
        # In our interleaved storage (T[0::2] = L and T[1::2] = W), water temperature at spatial point i is T[2*i+1].
        # So for indices in notice_idx, water temperatures are:
        water_temps = T[2 * np.array(notice_idx) + 1]
        cold = np.where(water_temps < Tfrz)[0]
        new_indices = np.array(notice_idx)[cold]
        if new_indices.size > 0:
            h[new_indices] = -Cw / Lfice * (W[new_indices] - Tfrz)
            W[new_indices] = Tfrz
    else:
        # No sea ice: update temperature from linear solve T = invM * r.
        T = invM @ r  # using the @ operator for matrix multiplication
        L = T[0::2]
        W = T[1::2]

    # --- Output saving ---
    # Check if the current time step is one of the output times.
    if n == n_out[idx_out]:
        L_out[:, idx_out] = L  # save current L (land temperatures)
        W_out[:, idx_out] = W  # save current W (water temperatures)
        if ice_model:
            h_out[:, idx_out] = h  # save current ice thickness
        idx_out += 1
        # (Make sure that n_out is sorted in increasing order.)

# --- Post-processing after loop ---
n_final = idx_out  # total number of outputs saved
# Create an array 'days' that counts from nstepinyear-1 down to 0.
days = np.arange(nstepinyear - 1, -1, -1)
tt = n_final - days  # this replicates: tt = n - days; in MATLAB

# Use tt to select days from the saved arrays.
thedays = np.array(day)[tt]
Lann = L_out[:, tt]
Wann = W_out[:, tt]

# Optionally, save the output to a .mat file (using scipy.io.savemat) or other format.
# For example:
# from scipy.io import savemat
# savemat('temperatures.mat', {'Lann': Lann, 'Wann': Wann, 'thedays': thedays})

# --- Plot the output ---
plotoutput()  # Assumes plotoutput() performs the plotting similar to MATLAB.


IndexError: boolean index did not match indexed array along axis 0; size of axis is 360 but size of corresponding boolean axis is 359